# 大语言模型微调

In [6]:
import os

# 设置代理
os.environ['http_proxy'] = 'http://127.0.0.1:7893'
os.environ['https_proxy'] = 'http://127.0.0.1:7893'
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7893'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7893'
os.environ['no_proxy'] = '127.0.0.1,localhost'
os.environ['NO_PROXY'] = '127.0.0.1,localhost'

# 验证
print(os.environ.get('http_proxy'))

http://127.0.0.1:7893


In [ ]:
from huggingface_hub import login

# 使用你的 token 登录
login()
print("Successfully logged in to Hugging Face!")

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Successfully logged in to Hugging Face!


In [8]:
from transformers import AutoTokenizer

# 使用你原来的模型
# Mistral 需要 token（通过上面的 login 已经提供）
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Qwen 需要 trust_remote_code=True 参数
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

# SmolLM 是开放模型
smol_tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"},
]

print("=" * 50)
print("Mistral Chat Template:")
print("=" * 50)
mistral_chat = mistral_tokenizer.apply_chat_template(messages, tokenize=False)
print(mistral_chat)

print("\n" + "=" * 50)
print("Qwen Chat Template (手动格式化):")
print("=" * 50)
# Qwen 使用特殊的格式，不使用 chat template
# 格式：<|im_start|>system\n{content}<|im_end|>\n<|im_start|>user\n{content}<|im_end|>\n<|im_start|>assistant\n
qwen_chat = ""
for message in messages:
    role = message["role"]
    content = message["content"]
    qwen_chat += f"<|im_start|>{role}\n{content}<|im_end|>\n"
qwen_chat += "<|im_start|>assistant\n"
print(qwen_chat)

print("\n" + "=" * 50)
print("SmolLM Chat Template:")
print("=" * 50)
smol_chat = smol_tokenizer.apply_chat_template(messages, tokenize=False)
print(smol_chat)

print("\n" + "=" * 50)
print("检查各个 tokenizer 是否有 chat_template:")
print("=" * 50)
print(f"Mistral has chat_template: {mistral_tokenizer.chat_template is not None}")
print(f"Qwen has chat_template: {qwen_tokenizer.chat_template is not None}")
print(f"SmolLM has chat_template: {smol_tokenizer.chat_template is not None}")

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-7B-Chat:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Mistral Chat Template:
<s> [INST] You are a helpful assistant.

Hello! [/INST]

Qwen Chat Template (手动格式化):
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant


SmolLM Chat Template:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello!<|im_end|>


检查各个 tokenizer 是否有 chat_template:
Mistral has chat_template: True
Qwen has chat_template: False
SmolLM has chat_template: True


In [16]:
# 测试工具调用 - 只有支持的模型才能处理 tool 角色
messages_with_tools = [
    {"role": "user", "content": "What's 123 * 456?"},
    {"role": "assistant", "content": "123 * 456 = 56,088"},
]

# Mistral 不支持 tool 角色，使用普通对话格式
mistral_chat = mistral_tokenizer.apply_chat_template(messages_with_tools, tokenize=False)
smol_chat = smol_tokenizer.apply_chat_template(messages_with_tools, tokenize=False)

print("Mistral格式：")
print(mistral_chat)
print("\nSmolLM格式：")
print(smol_chat)

Mistral格式：
<s> [INST] What's 123 * 456? [/INST] 123 * 456 = 56,088</s>

SmolLM格式：
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What's 123 * 456?<|im_end|>
<|im_start|>assistant
123 * 456 = 56,088<|im_end|>



In [20]:
qwen3_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", trust_remote_code=True)

messages = [
    {
        "role": "system",
        "content": "You are an AI assistant that can use tools. Available tools: calculator, weather_api",
    },
    {"role": "user", "content": "What's 123 * 456 and is it raining in Paris?"},
    {
        "role": "assistant",
        "content": "Let me help you with that.",
        "tool_calls": [
            {
                "id": "call_1",
                "type": "function",
                "function": {
                    "name": "calculator",
                    "arguments": '{"operation": "multiply", "x": 123, "y": 456}'
                }
            },
            {
                "id": "call_2", 
                "type": "function",
                "function": {
                    "name": "weather_api",
                    "arguments": '{"city": "Paris", "country": "France"}'
                }
            }
        ],
    },
    {"role": "tool", "tool_call_id": "call_1", "content": "56088"},
    {
        "role": "tool",
        "tool_call_id": "call_2",
        "content": "{'condition': 'rain', 'temperature': 15}",
    },
]

qwen3_chat = qwen3_tokenizer.apply_chat_template(messages, tokenize=False)
print(qwen3_chat)

<|im_start|>system
You are an AI assistant that can use tools. Available tools: calculator, weather_api<|im_end|>
<|im_start|>user
What's 123 * 456 and is it raining in Paris?<|im_end|>
<|im_start|>assistant
Let me help you with that.
<tool_call>
{"name": "calculator", "arguments": {"operation": "multiply", "x": 123, "y": 456}}
</tool_call>
<tool_call>
{"name": "weather_api", "arguments": {"city": "Paris", "country": "France"}}
</tool_call><|im_end|>
<|im_start|>user
<tool_response>
56088
</tool_response>
<tool_response>
{'condition': 'rain', 'temperature': 15}
</tool_response><|im_end|>

